In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os 
os.chdir("file_path")
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
best_cnn_classifier_acc_58.pth	final_model_cifar10.pth
best_cnn_classifier.pth		mini-imagenet-cache-dataset.pkl
best_cnn_miniImagenet.pth	__pycache__
CNN_Cifar10.ipynb		TransferLearning.ipynb
CNN_miniImagenet.ipynb		utils.py
data


In [2]:
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
import pickle
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
import torch.nn.functional as F

class CNN_Net(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [7]:
def train():
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # forward + backward + optimize
            outputs = model(inputs.to(device)).to(device)
            loss = criterion(outputs, labels.to(device))
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

In [8]:
input_shape = 32 #(32x32)
input_channels = 3
num_classes = 10
num_epochs = 5
learning_rate = 0.001
momentum=0.9

In [9]:
model = CNN_Net(input_channels, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
train()
torch.save(model.state_dict(), 'final_model_cifar10.pth')

[1,  2000] loss: 2.203
[1,  4000] loss: 1.842
[1,  6000] loss: 1.645
[1,  8000] loss: 1.560
[1, 10000] loss: 1.485
[1, 12000] loss: 1.472
[2,  2000] loss: 1.364
[2,  4000] loss: 1.361
[2,  6000] loss: 1.342
[2,  8000] loss: 1.296
[2, 10000] loss: 1.309
[2, 12000] loss: 1.279
[3,  2000] loss: 1.191
[3,  4000] loss: 1.193
[3,  6000] loss: 1.187
[3,  8000] loss: 1.198
[3, 10000] loss: 1.170
[3, 12000] loss: 1.142
[4,  2000] loss: 1.081
[4,  4000] loss: 1.090
[4,  6000] loss: 1.097
[4,  8000] loss: 1.103
[4, 10000] loss: 1.079
[4, 12000] loss: 1.074
[5,  2000] loss: 1.014
[5,  4000] loss: 1.002
[5,  6000] loss: 1.015
[5,  8000] loss: 1.015
[5, 10000] loss: 1.027
[5, 12000] loss: 1.008
Finished Training


In [10]:
model = CNN_Net(input_channels, num_classes).to(device)
model.load_state_dict(torch.load('final_model_cifar10.pth'))
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %
